In [19]:
from dfply import *
import pandas as pd
import numpy as np

In [8]:
manu = pd.read_csv("C:/3-MSBA - Summer/Programming/Week 8/manufacturers.csv")
prod = pd.read_csv("C:/3-MSBA - Summer/Programming/Week 8/products.csv")

In [22]:
prod >> head(5)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
3,4,Floppy disk,5,6
4,5,Monitor,240,1


In [21]:
prod >> distinct(X.name)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
3,4,Floppy disk,5,6
4,5,Monitor,240,1
5,6,DVD drive,180,2
6,7,CD drive,90,2
7,8,Printer,270,3
8,9,Toner cartridge,66,3
9,10,DVD burner,180,2


In [23]:
manu >> head(5)

,mfr_id,mfr_name
0,1,Sony
1,2,Creative Labs
2,3,Hewlett-Packard
3,4,Iomega
4,5,Fujitsu


# Filter for products with price > 100

In [24]:
prod >> mask(X.price > 100)

,prd_id,name,price,mfr_id
0,1,Hard drive,240,5
1,2,Memory,120,6
2,3,ZIP drive,150,4
4,5,Monitor,240,1
5,6,DVD drive,180,2
7,8,Printer,270,3
9,10,DVD burner,180,2


# Merge with manufacturers datset

In [26]:
# Names have to consistent to join
prod_manu = prod >> left_join(manu, by = "mfr_id")
prod_manu

,prd_id,name,price,mfr_id,mfr_name
0,1,Hard drive,240,5,Fujitsu
1,2,Memory,120,6,Winchester
2,3,ZIP drive,150,4,Iomega
3,4,Floppy disk,5,6,Winchester
4,5,Monitor,240,1,Sony
5,6,DVD drive,180,2,Creative Labs
6,7,CD drive,90,2,Creative Labs
7,8,Printer,270,3,Hewlett-Packard
8,9,Toner cartridge,66,3,Hewlett-Packard
9,10,DVD burner,180,2,Creative Labs


In [33]:
#Filter for manufacturers with average product price of greater than 150
manu_150 = prod_manu >> group_by(X.mfr_id,X.mfr_name) >> summarize(avg_price = mean(X.price)) >> mask(X.avg_price >= 150)
# The columns mentioned in the group_by statement become the columns of the dataframe
manu_150.dtypes

mfr_name      object
mfr_id         int64
avg_price    float64
dtype: object

In [34]:
manu_150

,mfr_name,mfr_id,avg_price
0,Sony,1,240.0
1,Creative Labs,2,150.0
2,Hewlett-Packard,3,168.0
3,Iomega,4,150.0
4,Fujitsu,5,240.0


In [71]:
baseball = pd.read_csv("C:/3-MSBA - Summer/Programming/Week 8/baseball.csv",names = ["Player_Name","Team_Name","Bats","Hits"])
baseball >> head(5)

,Player_Name,Team_Name,Bats,Hits
0,A. J. Pierzynski,White Sox,479,133
1,Aaron Hill,Diamondbacks,609,184
2,Adam Dunn,White Sox,539,110
3,Adam Jones,Orioles,648,186
4,Adam LaRoche,Nationals,571,155


In [72]:
# Filter for teams with 7 or more number of players in it
# Create a column with number of players by team in the dataframe
# Its a must to pass a column name to n or n_distinct - Need to pass the granularity of the table to obtain frequency
baseball >> group_by(X.Team_Name) >> summarize(player_count = n_distinct(X.Player_Name))

# Filter for player_count >= 7
baseball >> group_by(X.Team_Name) >> summarize(player_count = n_distinct(X.Player_Name)) >> mask(X.player_count >= 7)


,Team_Name,player_count
0,Angels,8
6,Cardinals,7
20,Rangers,7
26,Tigers,7
27,Twins,7
28,White Sox,9
29,Yankees,7


In [79]:
# Compute the batting average, Batting Average = Team_total_hits/ Teams total at bat
baseball = baseball >> group_by(X.Team_Name) >> summarize(bat_avg = X.Hits.sum()/X.Bats.sum()) >> ungr

# Printing the dataframe after groupby prints the entire dataframe - 
#I have posted an issue in Github regarding the same. Hence, refrain from using head after group_by

In [81]:
# Filter for teams with batting average greater than .2 and less than .25
baseball >> mask(X.bat_avg >= 0.2,X.bat_avg <= 0.25)

,Team_Name,bat_avg
12,Mariners,0.242055
18,Phillies,0.250000
21,Rays,0.240749
23,Reds,0.249316
